In [1]:
%load_ext autoreload


Aprox 60 celdas por arista en malla original 40_h_repaired de 40 de arista -> 0.6 de tamaño de malla. Se supone que es 1.

115 tamaño -> 40 um

40 tamaño -> 14 um

14 um -> 0.23 um de tamaño de elemento de malla. 

Cosas a cambiar:
    1. Tamaño de elemento de malla, probar con mallas más finas/gruesas.
    2. Box meshes de distintos tamaños
    3. Distintos combos preconditioner+solver
    
Qué está funcionando ahora:
    1. 

In [2]:
__author__ = 'biherrera'
__email__ = 'biherrera@uc.cl'

%autoreload

import sys
import os
import time
import numpy as np
import matplotlib.pyplot as plt
sys.path.append(os.getcwd()[:-6])
print("Relative path: ", os.getcwd()[:-6])
import dolfin
from fenics import *
from src.model import PerfusionGasExchangeModel
from src.params import params

Relative path:  /home/bnherrerac/alveolar-perfusion-transport-modeling


In [ ]:
max_dims = [39.894161224365234, 39.895729064941406, 39.89208984375]
min_dims = [0.09939099848270416, 0.09558500349521637, 0.1048400029540062]

def run_model(name, fname, solver, preconditioner, n_jobs=15):
    print(f"Running model on {n_jobs} jobs.")
    print("Imported src files")
    print("Starting...")
    folder = fname + "/" + name
    path = os.path.join("../../../results-data/", folder)
    model = PerfusionGasExchangeModel(folder_path=path, params=params, solver='gmres', f_dim = 2, vecf_dim=1)
    print("Model initialised")
    
    model.import_mesh(
        os.path.join("../../../raw-data/40_h_repaired", "40_h_repaired.xdmf"), type="xdmf", 
        periodic=False, max_dims=max_dims, min_dims=min_dims, tol=0.1
    )
    print("Mesh imported")
    
    model.mesh = dolfin.refine(model.mesh)
    print("Mesh refined")
    
    print("Starting (P) simulation")
    model.sim_p(save=True, meshtype="tkd")
    print("(P) simulation done")
    
    print("Starting (T) simulation")
    x = model.sim_t(hb=False, save=True, solver="bicgstab")
    print("Finished (linear) guess generation")
    solution = model.sim_t(hb=True, save=True, guess=x, solver=solver, preconditioner=preconditioner)
    print("Done")

In [8]:
# Single-run cell
print("Imported src files")
print("Starting...")
folder = "box_mesh_on_cluster"
path = os.path.join("../results-data", folder)
print(path)
model = PerfusionGasExchangeModel(folder_path=path, params=params, solver='mumps', f_dim = 2, vecf_dim=1)
print("Model initialised")
# max_dimsh = [12.005523, 12.005658, 12.005917]
# min_dimsh = [1.994504, 1.993421, 1.994056]
max_dimsh = [5,5,5]
min_dimsh = [0,0,0]

model.import_mesh(
    os.path.join("../raw-data/small_cube", "small_cube.xdmf"), type="xdmf", 
    periodic=False, max_dims = max_dimsh, min_dims = min_dimsh, tol=0.1
)
print("Mesh imported")
# model.mesh = dolfin.refine(model.mesh)
model.mesh = box_mesh
# print("Mesh refined")
print("Starting (P) simulation")
t_1 = time.time()
model.sim_p(save=True, meshtype="tkd")
print("Time elapsed: ", time.time()-t_1)
print("(P) simulation done")

Imported src files
Starting...
../results-data/box_mesh_on_cluster
Model initialised
Mesh imported
Coordinates have shape  (879, 3)
Mesh imported
Starting (P) simulation
began sim_p
Instancing boundaries
gamma_in.dir_min =  0
gamma_in.dir_max =  5
gamma_out.dir_min =  0
gamma_out.dir_max =  5
gamma_air.dir_min_y =  0
gamma_air.dir_max_y =  5
gamma_air.dir_min_z =  0
gamma_air.dir_max_z =  5
boundaries instanced
self.f_dim = 2
self.vecf_dim = 1
P problem solved
u problem solved
Saving solutions
saved
Time elapsed:  83.204030752182
(P) simulation done


In [ ]:
%autoreload
print("Starting (T) simulation")
t1 = time.time()
x = model.sim_t(hb=False, save=True, solver="bicgstab")
print(f"Time elapsed: {time.time()-t1}")
print("Finished (linear) guess generation")

Starting (T) simulation
began sim_t, with solver =  bicgstab
Solving with solver = bicgstab and without preconditioner.


In [7]:
%autoreload
t1 = time.time()
solution = model.sim_t(
    hb=True, save=False, guess=x, 
    solver="gmres")
print(f"Time elapsed: {time.time()-t1}")
print("Done")

began sim_t, with solver =  gmres
Solving with solver = gmres and without preconditioner.


RuntimeError: 

*** -------------------------------------------------------------------------
*** DOLFIN encountered an error. If you are not able to resolve this issue
*** using the information listed below, you can ask for help at
***
***     fenics-support@googlegroups.com
***
*** Remember to include the error message listed below and, if possible,
*** include a *minimal* running example to reproduce the error.
***
*** -------------------------------------------------------------------------
*** Error:   Unable to solve linear system using Krylov iteration.
*** Reason:  Unknown preconditioner "sorrr". Use list_krylov_solver_preconditioners() to list available preconditioners().
*** Where:   This error was encountered inside KrylovSolver.cpp.
*** Process: 0
*** 
*** DOLFIN version: 2019.2.0.dev0
*** Git changeset:  unknown
*** -------------------------------------------------------------------------


In [ ]:
if not has_linear_algebra_backend("PETSc") and not has_linear_algebra_backend("Tpetra"):
    info("DOLFIN has not been configured with Trilinos or PETSc. Exiting.")
    exit()

if not has_krylov_solver_preconditioner("amg"):
    info("Sorry, this demo is only available when DOLFIN is compiled with AMG "
         "preconditioner, Hypre or ML.")
    exit()

if has_krylov_solver_method("minres"):
    krylov_method = "minres"
elif has_krylov_solver_method("tfqmr"):
    krylov_method = "tfqmr"
else:
    info("Default linear algebra backend was not compiled with MINRES or TFQMR "
         "Krylov subspace method. Terminating.")
    exit()

In [ ]:
print(dolfin.__version__)
print(os.path.dirname(dolfin.__file__))

In [ ]:
list_lu_solver_methods()
list_krylov_solver_methods()
list_linear_solver_methods() # lista todo
list_krylov_solver_preconditioners()